In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

In [2]:
# One hot encode for Nominal ONLY
# Functions for repeated graphs
# Check if across categories if sale price varies
# Don't use correlation for categorical features
# 5 outliers, 3 are in test set. Need to remove 2 in train set
# if want to remove, need to prove via stdev
# Can choose to drop one column if there is high collinearity for 2 variables

In [43]:
data = pd.read_csv('./datasets/train.csv')
# 23 Nominal, 23 Ordinal, 14 Discrete, 20 Continuous

ord_features = [
    'Lot Shape',
    'Utilities',
    'Land Slope',
    'Overall Qual',
    'Overall Cond',
    'Exter Qual',
    'Exter Cond',
    'Bsmt Qual',
    'Bsmt Cond',
    'Bsmt Exposure',
    'BsmtFin Type 1',
    'BsmtFin Type 2',
    'Heating QC',
    'Electrical',
    'Kitchen Qual',
    'Functional',
    'Fireplace Qu',
    'Garage Finish',
    'Garage Qual',
    'Garage Cond',
    'Paved Drive',
    'Pool QC',
    'Fence'    
]

nom_features = [
    'PID', #** Exclude PID as nom observation ids
    'MS SubClass',
    'MS Zoning',
    'Street',
    'Alley',
    'Land Contour',
    'Lot Config',
    'Neighborhood',
    'Condition 1',
    'Condition 2',
    'Bldg Type',
    'House Style',
    'Roof Style',
    'Roof Matl',
    'Exterior 1st',
    'Exterior 2nd',
    'Mas Vnr Type',
    'Foundation',
    'Heating',
    'Central Air',
    'Garage Type',
    'Misc Feature',
    'Sale Type',
#     'Sale Condition'
]

cont_features = [
    'Lot Frontage',
    'Lot Area',
    'Mas Vnr Area',
    'BsmtFin SF 1',
    'BsmtFin SF 2',
    'Bsmt Unf SF',
    'Total Bsmt SF',
    '1st Flr SF',
    '2nd Flr SF',
    'Low Qual Fin SF',
    'Gr Liv Area',
    'Garage Area',
    'Wood Deck SF',
    'Open Porch SF',
    'Enclosed Porch',
    '3Ssn Porch',
    'Screen Porch',
    'Pool Area',
    'Misc Val',
    'SalePrice'
]


disc_features = [
    'Id',
    'Year Built',
    'Year Remod/Add',
    'Bsmt Full Bath',
    'Bsmt Half Bath',
    'Full Bath',
    'Half Bath',
    'Bedroom AbvGr',
    'Kitchen AbvGr',
    'TotRms AbvGrd',
    'Fireplaces',
    'Garage Yr Blt',
    'Garage Cars',
    'Mo Sold',
    'Yr Sold'
]


In [35]:
def check_nan_values(df, col_lst):
    ft_lst = []
    for feature in df.loc[:,(df[col_lst].isnull().sum() > 0).index].columns:
        if df[feature].isnull().values.any():
            ft_lst.append(feature)
    return ft_lst


In [45]:
check_nan_values(data,nom_features)


['Alley', 'Mas Vnr Type', 'Garage Type', 'Misc Feature']

In [46]:
# Clean nominal features
data['Alley'] = data['Alley'].fillna('NA')
data['Garage Type'] = data['Garage Type'].fillna('NA')
data['Misc Feature'] = data['Misc Feature'].fillna('NA')
data['Mas Vnr Type'] = data['Mas Vnr Type'].map(lambda x: 'CBlock' if pd.isnull(x) else x)

In [48]:
data.loc[:,['Alley','Mas Vnr Type','Garage Type','Misc Feature']].describe().T

,count,unique,top,freq
Alley,2051,3,NA,1911
Mas Vnr Type,2051,5,None,1218
Garage Type,2051,7,Attchd,1213
Misc Feature,2051,6,NA,1986


In [ ]:
# Clean nominal features
# data['Alley'] = data['Alley'].map(lambda x: 'NA' if pd.isnull(x) else x)
# data['Mas Vnr Type'] = data['Mas Vnr Type'].map(lambda x: 'CBlock' if pd.isnull(x) else x)
# data['Garage Type'] = data['Garage Type'].map(lambda x: 'NA' if pd.isnull(x) else x)
# data['Misc Feature'] = data['Misc Feature'].map(lambda x: 'NA' if pd.isnull(x) else x )

In [ ]:
# data.loc[:,['Alley','Mas Vnr Type','Garage Type','Misc Feature']].describe().T

In [52]:
# Clean Ordinal features
for ft in check_nan_values(data,ord_features):
    data[ft] = data[ft].fillna('NA')

In [53]:
data.loc[:,['Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin Type 2','Fireplace Qu','Garage Finish','Garage Qual','Garage Cond','Pool QC','Fence']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Bsmt Qual       2051 non-null   object
 1   Bsmt Cond       2051 non-null   object
 2   Bsmt Exposure   2051 non-null   object
 3   BsmtFin Type 1  2051 non-null   object
 4   BsmtFin Type 2  2051 non-null   object
 5   Fireplace Qu    2051 non-null   object
 6   Garage Finish   2051 non-null   object
 7   Garage Qual     2051 non-null   object
 8   Garage Cond     2051 non-null   object
 9   Pool QC         2051 non-null   object
 10  Fence           2051 non-null   object
dtypes: object(11)
memory usage: 176.4+ KB


In [ ]:
# Clean ordinal features
data['Bsmt Qual'] = data['Bsmt Qual'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['Bsmt Cond'] = data['Bsmt Cond'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['Bsmt Exposure'] = data['Bsmt Exposure'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['BsmtFin Type 1'] = data['BsmtFin Type 1'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['BsmtFin Type 2'] = data['BsmtFin Type 2'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['Fireplace Qu'] = data['Fireplace Qu'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['Garage Finish'] = data['Garage Finish'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['Garage Qual'] = data['Garage Qual'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['Garage Cond'] = data['Garage Cond'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['Pool QC'] = data['Pool QC'].map(lambda x: 'NA' if pd.isnull(x) else x )
data['Fence'] = data['Fence'].map(lambda x: 'NA' if pd.isnull(x) else x )

In [54]:
data.loc[:,['Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin Type 2','Fireplace Qu','Garage Finish','Garage Qual','Garage Cond','Pool QC','Fence']].describe().T

,count,unique,top,freq
Bsmt Qual,2051,6,TA,887
Bsmt Cond,2051,6,TA,1834
Bsmt Exposure,2051,5,No,1339
BsmtFin Type 1,2051,7,GLQ,615
BsmtFin Type 2,2051,7,Unf,1749
Fireplace Qu,2051,6,NA,1000
Garage Finish,2051,4,Unf,849
Garage Qual,2051,6,TA,1832
Garage Cond,2051,6,TA,1868
Pool QC,2051,5,NA,2042


In [ ]:
data.loc[:,['Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin Type 2','Fireplace Qu','Garage Finish','Garage Qual','Garage Cond','Pool QC','Fence']].info()

In [ ]:
check_nan_values(data,cont_features)


In [ ]:
# Clean continuous features
data['Lot Frontage'] = data['Lot Frontage'].map(lambda x: 0 if pd.isnull(x) else x )
data['Mas Vnr Area'] = data['Mas Vnr Area'].map(lambda x: 0 if pd.isnull(x) else x )
data['BsmtFin SF 1'] = data['BsmtFin SF 1'].map(lambda x: 0 if pd.isnull(x) else x )
data['BsmtFin SF 2'] = data['BsmtFin SF 2'].map(lambda x: 0 if pd.isnull(x) else x )
data['Bsmt Unf SF'] = data['Bsmt Unf SF'].map(lambda x: 0 if pd.isnull(x) else x )
data['Total Bsmt SF'] = data['Total Bsmt SF'].map(lambda x: 0 if pd.isnull(x) else x )
data['Garage Area'] = data['Garage Area'].map(lambda x: 0 if pd.isnull(x) else x )

In [ ]:
data.loc[:,['Lot Frontage',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Garage Area']].info()

In [ ]:
check_nan_values(data,disc_features)

In [ ]:
data['Bsmt Full Bath'] = data['Bsmt Full Bath'].map(lambda x: 0 if pd.isnull(x) else x )
data['Bsmt Half Bath'] = data['Bsmt Half Bath'].map(lambda x: 0 if pd.isnull(x) else x )
data['Garage Yr Blt'] = data['Garage Yr Blt'].map(lambda x: 0 if pd.isnull(x) else x )
data['Garage Cars'] = data['Garage Cars'].map(lambda x: 0 if pd.isnull(x) else x )

In [ ]:
data.loc[:,['Bsmt Full Bath', 'Bsmt Half Bath', 'Garage Yr Blt', 'Garage Cars']].info()

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
data['Garage Yr Blt'].describe()   # Noted typo of 2207. Deduced to be 2007 instead
data.loc[data['Garage Yr Blt'] == 2207, 'Garage Yr Blt'] = 2007

In [ ]:
data['Central Air'] = data['Central Air'].map(lambda x: 1 if x == 'Y' else 0)

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data=data.corr()[['SalePrice']].sort_values(ascending=False, by='SalePrice'), annot=True, cmap='coolwarm')

In [ ]:
data.to_csv('./datasets/cleaned_data.csv')